<a href="https://colab.research.google.com/github/rfflpllcn/ml/blob/master/imdb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [40]:
# Install Kaggle library
!pip install -q kaggle

# Colab library to upload files to notebook
from google.colab import files
import pandas as pd

from keras.preprocessing.text import Tokenizer


Using TensorFlow backend.


In [0]:
def tokenizer(text):
    """ 
    Simple Parser converting each document to lower-case, then
    removing the breaks for new lines and finally splitting on the
    whitespace
    from: https://github.com/dandxy89/DeepLearningReading/blob/master/EmbeddingKeras/imdb_embedding_w2v.py
    """
    text = [document.lower().replace('\n', '').split() for document in text]
    return text

In [3]:
# Upload kaggle API key file
uploaded = files.upload()

Saving kaggle.json to kaggle.json


In [0]:
!mkdir ~/.kaggle
!touch ~/.kaggle/kaggle.json
!echo '{"username":"michelegiambono","key":"ca6243facbe66d92bf3688aec1a2fd8c"}' > ~/.kaggle/kaggle.json
!chmod 600 /root/.kaggle/kaggle.json

# Downlaod data 
!kaggle datasets download -d utathya/imdb-review-dataset -p /content
!unzip \*.zip

In [0]:
def get_data(df):
  # return test and train data with labels
  train = df[df['type'] == 'train']
  test = df[df['type'] == 'test']
  
  train_labels = train['label'].values == 'pos'
  test_labels = test['label'].values == 'pos'

  return (train['review'].values, train_labels), (test['review'].values, test_labels)

In [0]:
d = pd.read_csv('imdb_master.csv', encoding = "ISO-8859-1")
(train_data, train_labels), (test_data, test_labels) = get_data(d)

In [3]:
t = Tokenizer(num_words=10000)
t.fit_on_texts(train_data.tolist()+test_data.tolist())

NameError: ignored

In [2]:
train_data_encoded = t.texts_to_matrix(train_data)

# sorted(t.word_counts.items(), key=lambda x: x[1])[::-1]

# t.document_count
# max([v for k, v in t.word_index.items()])
# t.word_docs

NameError: ignored

In [1]:
train_data_encoded.shape

NameError: ignored

In [54]:
t.word_counts

OrderedDict([('story', 46380),
             ('of', 583306),
             ('a', 650973),
             ('man', 21932),
             ('who', 82608),
             ('has', 67058),
             ('unnatural', 201),
             ('feelings', 1559),
             ('for', 176751),
             ('pig', 377),
             ('starts', 4846),
             ('out', 68433),
             ('with', 177148),
             ('opening', 3854),
             ('scene', 22015),
             ('that', 275653),
             ('is', 425423),
             ('terrific', 1664),
             ('example', 5075),
             ('absurd', 1099),
             ('comedy', 13297),
             ('formal', 99),
             ('orchestra', 195),
             ('audience', 8371),
             ('turned', 3783),
             ('into', 36223),
             ('an', 86466),
             ('insane', 1025),
             ('violent', 2195),
             ('mob', 781),
             ('by', 90147),
             ('the', 1344272),
             ('crazy', 2511